# Preprocessing the ASL Alphabet Dataset

This notebook demonstrates how to preprocess the ASL Alphabet Dataset, including loading, resizing, normalizing, label encoding, and augmenting images.

## Import Required Libraries

This code cell imports all the necessary libraries for data preprocessing, including TensorFlow, NumPy, and scikit-learn components.

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder

# Debugging code to check the directory structure and contents
base_path = "C:/Users/User/OneDrive - belgiumcampus.ac.za/Marius_Grassman/ML-Sign-Language-Recognition/data/"
print("Contents of data directory:")
for item in os.listdir(base_path):
    item_path = os.path.join(base_path, item)
    print(f"- {item} {'(directory)' if os.path.isdir(item_path) else '(file)'}")

train_path = os.path.join(base_path, "asl_alphabet_train")
if os.path.exists(train_path):
    print(f"\nContents of {train_path}:")
    for item in os.listdir(train_path):
        item_path = os.path.join(train_path, item)
        print(f"- {item} {'(directory)' if os.path.isdir(item_path) else '(file)'}")

Contents of data directory:
- asl_alphabet_test (directory)
- asl_alphabet_train (directory)
- README.md (file)

Contents of C:/Users/User/OneDrive - belgiumcampus.ac.za/Marius_Grassman/ML-Sign-Language-Recognition/data/asl_alphabet_train:
- asl_alphabet_train (directory)


## Define Dataset Loading Function

This function will load the ASL dataset from the specified directory. It will be implemented to read images and their corresponding labels from the folder structure.

In [2]:
def load_dataset(data_dir, max_images_per_class=50):
    """
    Load a limited number of images per class from the ASL dataset directory.
    
    Args:
        data_dir (str): Path to the dataset directory
        max_images_per_class (int): Maximum images to load per class
        
    Returns:
        tuple: (images, labels) - Arrays of images and their corresponding labels
    """
    images = []
    labels = []
    for label_dir in os.listdir(data_dir):
        label_path = os.path.join(data_dir, label_dir)
        if os.path.isdir(label_path):
            count = 0
            for image_file in os.listdir(label_path):
                if count >= max_images_per_class:
                    break
                image_path = os.path.join(label_path, image_file)
                try:
                    image = tf.keras.preprocessing.image.load_img(image_path)
                    images.append(tf.keras.preprocessing.image.img_to_array(image))
                    labels.append(label_dir)
                    count += 1
                except Exception as e:
                    continue
    return np.array(images), np.array(labels)

## Define Image Resizing Function

This function resizes all images to a consistent size (224x224 pixels) to ensure uniform input for the neural network.

In [3]:
def resize_images(images, target_size=(224, 224)):
    """
    Resize images to a consistent target size.
    
    Args:
        images (array): Array of images to resize
        target_size (tuple): Target size (height, width)
        
    Returns:
        np.array: Array of resized images
    """
    resized_images = [tf.image.resize(image, target_size) for image in images]
    return np.array(resized_images)

## Define Image Normalization Function

This function normalizes pixel values from the range [0, 255] to [0, 1] to improve neural network training performance.

In [4]:
def normalize_images(images):
    """
    Normalize pixel values from [0, 255] to [0, 1].
    
    Args:
        images (array): Array of images to normalize
        
    Returns:
        np.array: Array of normalized images
    """
    normalized_images = images / 255.0
    return normalized_images

## Define Label Encoding Function

This function converts text labels (A, B, C, etc.) into numerical values that can be used by the neural network.

In [5]:
def encode_labels(labels):
    """
    Encode text labels into numerical values.
    
    Args:
        labels (array): Array of text labels (e.g., ['A', 'B', 'C'])
        
    Returns:
        array: Encoded numerical labels
    """
    encoder = LabelEncoder()
    encoded_labels = encoder.fit_transform(labels)
    return encoded_labels

## Define Data Augmentation Function

This function applies data augmentation techniques (rotation, shifting, flipping) to increase the diversity of the training data and improve model generalization.

In [6]:
def augment_images(images):
    """
    Apply data augmentation to images.
    
    Args:
        images (array): Array of images to augment
        
    Returns:
        np.array: Array of augmented images
    """
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True
    )
    augmented_images = [datagen.random_transform(image) for image in images]
    return np.array(augmented_images)

## Demonstrate Complete Preprocessing Pipeline

This function demonstrates the complete preprocessing pipeline by applying all the preprocessing steps to a sample dataset.

In [7]:
def demonstrate_preprocessing(data_dir):
    """
    Demonstrate the complete preprocessing pipeline on a sample dataset.
    
    Args:
        data_dir (str): Path to the dataset directory
        
    Returns:
        tuple: Processed images and encoded labels ready for training
    """
    # Load the dataset
    sample_images, sample_labels = load_dataset(data_dir)
    
    # Apply preprocessing steps
    resized_images = resize_images(sample_images)
    normalized_images = normalize_images(resized_images)
    augmented_images = augment_images(normalized_images)
    encoded_labels = encode_labels(sample_labels)
    
    print("Preprocessing complete. Sample images ready for training.")
    print(f"Processed {len(augmented_images)} images")
    print(f"Image shape: {augmented_images[0].shape}")
    print(f"Number of unique labels: {len(np.unique(encoded_labels))}")
    
    return augmented_images, encoded_labels

# Example usage
# Use the correct path to the actual image folders, not the parent directory
DATA_DIR = "C:/Users/User/OneDrive - belgiumcampus.ac.za/Marius_Grassman/ML-Sign-Language-Recognition/data/asl_alphabet_train/asl_alphabet_train/"
processed_images, processed_labels = demonstrate_preprocessing(DATA_DIR)

Preprocessing complete. Sample images ready for training.
Processed 1450 images
Image shape: (224, 224, 3)
Number of unique labels: 29
